# Подготовка теста (3 балла)


В первой части нужно подготовить данные

In [1]:
!pip install --quiet sentencepiece datasets transformers

     |████████████████████████████████| 1.3 MB 4.9 MB/s 
     |████████████████████████████████| 451 kB 43.6 MB/s 
     |████████████████████████████████| 5.8 MB 22.0 MB/s 
     |████████████████████████████████| 182 kB 45.0 MB/s 
     |████████████████████████████████| 132 kB 23.5 MB/s 
     |████████████████████████████████| 212 kB 14.3 MB/s 
     |████████████████████████████████| 127 kB 35.1 MB/s 
     |████████████████████████████████| 7.6 MB 49.5 MB/s 


In [2]:
import random
import torch 
import numpy as np
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import train_test_split
import os

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

Скачаем архив с данными и распаковываем

In [3]:
!wget -nc https://atsky-data.s3.amazonaws.com/data.tar.gz

--2022-12-06 20:09:46--  https://atsky-data.s3.amazonaws.com/data.tar.gz
Resolving atsky-data.s3.amazonaws.com (atsky-data.s3.amazonaws.com)... 52.216.40.49, 52.217.165.113, 52.216.102.11, ...
Connecting to atsky-data.s3.amazonaws.com (atsky-data.s3.amazonaws.com)|52.216.40.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63420861 (60M) [application/x-gzip]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]  60.48M  47.3MB/s    in 1.3s    

2022-12-06 20:09:47 (47.3 MB/s) - ‘data.tar.gz’ saved [63420861/63420861]



In [4]:
!ls /content/

data.tar.gz  sample_data


In [5]:
!tar -zxf /content/data.tar.gz

In [6]:
!ls /content

books_txt  data.tar.gz	sample_data  vocab.txt


Данные сложены в папке `/content/books_txt`

Реализуйте метод, который соберет пути ко всем текстовыы файлы в директории

In [7]:
def get_all_files(load_directory):
    all_paths = []
    # TODO реализовать
    return all_paths

In [12]:
paths = get_all_files("/content/books_txt")
paths[:10]

['/content/books_txt/GARSHIN/fordays.txt',
 '/content/books_txt/GARSHIN/redflower.txt',
 '/content/books_txt/NIKITIN_I/stihi.txt',
 '/content/books_txt/DOBROLYUBOW/oblomov.txt',
 '/content/books_txt/DOBROLYUBOW/groza.txt',
 '/content/books_txt/DOBROLYUBOW/nakanune.txt',
 '/content/books_txt/GERCEN/gersen_byloe.txt',
 '/content/books_txt/PISAREW/pisarev1_10.txt',
 '/content/books_txt/PISAREW/pisarev1_9.txt',
 '/content/books_txt/PISAREW/pisarev1_14.txt']

In [9]:
# реализуйте функцию, которя читает файл
def read_file(path):
    ...
    # TODO: Your code here


In [ ]:
read_file(paths[0])

In [ ]:
def remove_consecutive_spaces(string):
    return ' '.join(string.split())
    
def chars(f, t):
    return list(map(chr, range(ord(f), ord(t)+1)))

valid_chars = set(chars('а', 'я') + ['ё'] + chars('a', 'z') + chars('0', '9') 
                  + list("<>(){}" + ".,\"!?;:-*—\'") )

# Приведите текст к нижнему регистру
# yберите лишние символы и пробелы идущие подряд
def clean_text(text):
    ...
    # TODO: Your code here
    return text;

In [ ]:
clean_text(read_file(paths[0]))

Напшите код, который по списку файлов читает тексты и нормализует их.

In [ ]:
def load_texts(paths):
    # TODO: Your code here
    ...        
    return texts

In [ ]:
paths = get_all_files("/content/books_txt")
paths.sort()

# Выделим небольшой процент данных для теста
paths_train, paths_test = train_test_split(paths, test_size=0.03, random_state=12345)

texts_train = load_texts(paths_train)
texts_test = load_texts(paths_test)

In [ ]:
paths_test

Для токенизации используется готовое решение из библиотеки Huggingface

In [ ]:
from transformers import GPT2TokenizerFast
from tokenizers.decoders import ByteLevel,WordPiece
from transformers import BertTokenizerFast
decoder = WordPiece()

In [ ]:
tokenizer = BertTokenizerFast('/content/vocab.txt')

In [ ]:
pad_token_idx = 0
sep_token_idx = 3

In [ ]:
def encode(text):
    return tokenizer.encode(text, add_special_tokens=False) 

In [ ]:
ids = encode("я помню чудное мгновенье")
ids

[156, 2769, 2723, 539, 2646, 1881]

In [ ]:
tokenizer.convert_ids_to_tokens(ids)

['я', 'помню', 'чуд', '##ное', 'мгнов', '##енье']

In [ ]:
decoder.decode(tokenizer.convert_ids_to_tokens(ids))

'я помню чудное мгновенье'



Конкатенируйте все тексты в большую последовательность токенов


In [ ]:
def combine_texts(texts):
    # TODO: Your code here
    
    return tokens

In [ ]:
tokenized_train = combine_texts(texts_train)

  0%|          | 0/726 [00:00<?, ?it/s]

Посмотрим количество токенов, должно получиться примерно 35087577

In [ ]:
print(len(tokenized_train))

35087577


In [16]:
max_seq_len = 256

batch_size = 16

Далее нужно нарезать текст на части размером `max_seq_len + 1`. Полезно что бы части пересекались, для этого добавлен параметр `offset` 

In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts, offset):
        # TODO: Your code here

    def __getitem__(self, idx):
        # TODO: Your code here

    def __len__(self):
        # TODO: Your code here

In [ ]:
def collate_batch(batch):
    src_list = []
    for src in batch:        
        src_list.append(torch.tensor(src))

    return pad_sequence(src_list, padding_value=pad_token_idx, batch_first=True)

In [ ]:
train_data = TextDataset(texts_train, max_seq_len // 2)
train_dataloader = DataLoader(train_data, batch_size, shuffle=True, collate_fn=collate_batch)

  0%|          | 0/726 [00:00<?, ?it/s]

In [ ]:
test_data = TextDataset(texts_test, max_seq_len * 2) # Что бы не ждать будем использовать большой offset
test_dataloader = DataLoader(test_data, batch_size, collate_fn=collate_batch)

  0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
print(len(train_data))

274119


In [ ]:
batch = next(iter(train_dataloader))
print(batch.shape)

torch.Size([16, 257])


Ожидается размер torch. `torch.Size([16, 257])`
